In [1]:
%pip install opencv-python
%pip install numpy
import numpy as np
import cv2 as cv
import copy
import os

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Question 2 compression avec LZW

In [2]:
# Get data in this cell
messageList = []
dataPath = "/data TP1/textes/"
currDirectory = os.getcwd()
fileList = os.listdir(currDirectory + dataPath)
for file in fileList:
    filePath = currDirectory + dataPath + file
    if file.endswith(".txt"):
        with open(filePath, 'r') as f:
            messageList += [f.read()]
            
print(messageList)
message = "ABAABAABACABBABCDAADACABABAAABAABBABABAABAAB"

['AAAAAAAAAAAAAAAAABBABAAAAAAAAAAAABAAAAAAAAAAAAAABBAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAAA', 'AAAAABBBBBBBBBAAAAAAAAABBBBBBBCBBBBAAACACBCBBBBBBBBBAAAAAAAAACAAAABBBBAAAAABB', 'FOFOFOEOKOWKOOWONOOOOOOWOENFOEOOOJSODNOJFSOJKOOOOBV', 'Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nulla posuere ligula id nibh egestas, id sollicitudin nisi interdum. Integer a elit sed sem egestas interdum vitae id risus. ', 'Aliquam placerat dictum lectus vel pellentesque. Vestibulum semper dui lacus, sed commodo dolor maximus a. Praesent convallis in quam in posuere. Ut eleifend tortor a laoreet imperdiet. Sed pharetra risus sit amet velit cursus semper. Nam congue ac lectus sit amet laoreet. Integer quis lacinia turpis. Maecenas erat orci, porttitor ac ligula dapibus, interdum fringilla nunc. Proin auctor, arcu ac fermentum ultricies, erat justo laoreet nunc, sit amet lacinia lorem elit vel dolor. Aenean fermentum, mi sed iaculis laoreet, mauris nibh accumsan mauris, imperdiet pretium dui enim sit am

In [3]:
def encode(message : str):
    dictsymb =[message[0]]
    dictbin = ["{:b}".format(0)]
    nbsymboles = 1
    for i in range(1,len(message)):
        if message[i] not in dictsymb:
            dictsymb += [message[i]]
            dictbin += ["{:b}".format(nbsymboles)] 
            nbsymboles +=1    
    longueurOriginale = np.ceil(np.log2(nbsymboles))*len(message)   
    for i in range(nbsymboles):
        dictbin[i] = "{:b}".format(i).zfill(int(np.ceil(np.log2(nbsymboles))))  
        dictsymb.sort()
        dictionnaire = np.transpose([dictsymb,dictbin])
    basicdictsymb = copy.deepcopy(dictsymb)
    basicdictbin = copy.deepcopy(dictbin)
        
    #encode
    i=0
    messageCode = []
    longueur = 0
    while i < len(message):
        precsouschaine = message[i] #sous-chaine qui sera codé
        souschaine = message[i] #sous-chaine qui sera codé + 1 caractère (pour le dictionnaire)

        #Cherche la plus grande sous-chaine. On ajoute un caractère au fur et à mesure.
        while souschaine in dictsymb and i < len(message):
            i += 1
            precsouschaine = souschaine
            if i < len(message):  #Si on a pas atteint la fin du message
                souschaine += message[i]  

        #Codage de la plus grande sous-chaine à l'aide du dictionnaire  
        codebinaire = [dictbin[dictsymb.index(precsouschaine)]]
        messageCode += codebinaire
        longueur += len(codebinaire[0]) 
        #Ajout de la sous-chaine codé + symbole suivant dans le dictionnaire.
        if i < len(message):
            dictsymb += [souschaine]
            dictbin += ["{:b}".format(nbsymboles)] 
            nbsymboles +=1

        #Ajout de 1 bit si requis
        if np.ceil(np.log2(nbsymboles)) > len(messageCode[-1]):
            for j in range(nbsymboles):
                dictbin[j] = "{:b}".format(j).zfill(int(np.ceil(np.log2(nbsymboles))))
    return messageCode, basicdictsymb, basicdictbin

In [4]:
def decode(message : str, basicdictsymb, basicdictbin):    
    dictsymb = basicdictsymb
    dictbin = basicdictbin
    
    # decode the message
    i=0
    decodedMessage = ""
    nbsymboles = len(basicdictsymb)
    nbBitUsed = int(np.ceil(np.log2(nbsymboles)))
    while i < len(message):
        currSymb = dictsymb[dictbin.index(message[i])]
        decodedMessage += currSymb
        # add the new symbol to the dictionary
        nbsymboles += 1
        nbBitUsed = int(np.ceil(np.log2(nbsymboles)))
        dictbin += ["{:b}".format(nbsymboles)] 
        # add 1bit if required
        if 2**nbBitUsed > len(dictsymb):
            for j in range(nbsymboles):
                dictbin[j] = "{:b}".format(j).zfill(int(np.ceil(np.log2(nbsymboles))))
        if i+1 < len(message):
            try:
                nextSymb = dictsymb[dictbin.index(message[i+1])]
            except:
                nextSymb = currSymb
        
        dictsymb += [currSymb+nextSymb[0]]
        i+=1
    return decodedMessage
        
        

In [5]:
for message in messageList:
    print("Message original : ", message)
    encodedMessage = encode(message)
    print("Message encodé : ", encodedMessage[0])
    print("Message décodé : ", decode(encodedMessage[0], encodedMessage[1], encodedMessage[2]))

Message original :  AAAAAAAAAAAAAAAAABBABAAAAAAAAAAAABAAAAAAAAAAAAAABBAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAAA
Message encodé :  ['0', '10', '11', '100', '101', '010', '001', '0001', '0000', '1001', '0110', '0101', '1011', '1100', '1101', '01110', '01101', '01111', '10011', '01111']
Message décodé :  AAAAAAAAAAAAAAAAABBABAAAAAAAAAAAABAAAAAAAAAAAAAABBAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAAA
Message original :  AAAAABBBBBBBBBAAAAAAAAABBBBBBBCBBBBAAACACBCBBBBBBBBBAAAAAAAAACAAAABBBBAAAAABB
Message encodé :  ['00', '11', '011', '001', '110', '111', '0111', '0100', '1010', '0101', '1000', '0110', '0010', '1000', '00100', '00010', '00000', '01111', '01111', '01101', '01001', '01011', '10001', '01010', '10000', '11010', '00001']
Message décodé :  AAAAABBBBBBBBBAAAAAAAAABBBBBBBCBBBBAAACACBCBBBBBBBBBAAAAAAAAACAAAABBBBAAAAABB
Message original :  FOFOFOEOKOWKOOWONOOOOOOWOENFOEOOOJSODNOJFSOJKOOOOBV
Message encodé :  ['0011', '0111', '1011', '1011', '0010', '0111', '00101', '00111', '01010', '10001', '10010', '